In [74]:
import pandas as pd
import datetime

import bankInfoExtractor as bie
import taggedToTree as ttt
import subprocess
import timestring

In [72]:
def stoinfo(s):
    #TBD: Remove hard coding of paths
    HOME_PATH = 'set_to_home_path'
    PROJECT_PATH = HOME_PATH + 'set_to_where_syntaxnet_is_installed'
                                #Eg: /home/user/syntaxnet/models/syntaxnet

    CWD = PROJECT_PATH
    SD_PATH = CWD + '/syntaxnet'
    DEMO_CMD = 'demo.sh'
    
    cmd = 'echo "' + s + '" | ' + SD_PATH + '/' + DEMO_CMD + ' 2>/dev/null'
    
    sno = subprocess.check_output(cmd, shell=True, cwd=CWD)

    sno_st = '\n'.join(sno.split('\n')[2:]).rstrip()
    print sno_st

    print "-----"
    root = ttt.str_to_tree(sno_st)
#    ttt.print_node(root, 0)

#    print "--- clean up tree to remove unnecessary nodes---"
    root = bie.cleanTree(root)
#    ttt.print_node(root, 0)

#    print "Extracting (keyword,adj)"
    (keyword,adj) = bie.bankKeyWord(root)
#    print "keyword: " + keyword
#    print "adj: " + adj

#    print "Extracting Timeline"
    r = bie.timeLine(root)
    return (keyword, adj, r)

In [75]:
key, adj, r = stoinfo("average balance from january 2016 to july 2016")

balance NN ROOT
 +-- average JJ amod
 +-- from IN prep
     +-- 2016 CD pobj
     |   +-- january JJ amod
     +-- to IN prep
         +-- 2016 CD pobj
             +-- july RB advmod
-----
Tree form
 from
	 2016
		 january
	 to
		 2016
			 july
-----
2016 january to 2016 july
from 2016 january to 2016 july
From 01/01/16 00:00:00 to 07/01/16 00:00:00


In [57]:
print 'key=%s, adj=%s, r=%s'%(key,adj,r)

key=balance, adj=avg, r=From 01/01/16 00:00:00 to 02/01/16 00:00:00


In [77]:
df = pd.read_csv('citistmt.csv', header=1)

df.columns = ['Date', 'Description', 'Amount','Balance']
df.loc[:,'Balance'] = df.Balance.map(lambda x: ''.join(x.split(',')))
df.loc[:,'Balance'] = df.Balance.astype(float)

df.loc[:,'Date'] = pd.to_datetime(df['Date'])

df.set_index(['Date'], drop=True, inplace=True)
df.head()

,Description,Amount,Balance
Date,,,
2015-07-12,Beginning balance as of 07/12/2015,NaN,14568.67
2015-07-13,CHECK TO ED,-250.00,14318.67
2015-07-14,CASANERO INC DES:DIRECT DEP,7810.59,22129.26
2015-07-14,CHECK 4125,-150.00,21979.26
2015-07-15,MILAGRO INC:DIR DEP,2250.65,24229.91


In [78]:
df.tail(30)

,Description,Amount,Balance
Date,,,
2016-05-13,CASANERO INC DES:DIRECT DEP,7810.59,79430.67
2016-05-16,ATM WITHDRWL COLLIN TX,-1500.00,77930.67
2016-05-16,LEG INSURANCE,-124.65,77806.02
2016-05-20,Check 4163,-150.00,77656.02
2016-05-26,CASANERO INC DES:DIRECT DEP,7810.59,85466.61
2016-05-31,KABULI LOAN PAYMENT,-213.00,85253.61
2016-06-01,MILAGRO INC:DIR DEP,2250.65,87504.26
2016-06-01,Check 4164,-500.00,87004.26
2016-06-01,Online scheduled transfer to Savings Account,-200.00,86804.26


In [49]:
def get_balance(adj, r, tdf):
    """
    adj: avg, min, max, curr
    """
    start = datetime.date(r.start.year, r.start.month, r.start.day)
    end = datetime.date(r.end.year, r.end.month, r.end.day)
    if adj == 'curr':
        return tdf.iloc[-1].Balance
    if adj == 'avg':
        return tdf[start:end].Balance.mean()

def act(key, adj, r, tdf):
    if key == 'balance':
        if adj == 'curr':
            print "Current Balance: %0.2f"%get_balance(adj, r, tdf)
        elif adj == 'avg':
            print "Avg Balance for Timeline: %s: %0.2f"%(r, get_balance(adj, r, tdf))

def decipher(s, tdf):
    key, adj, r = stoinfo(s)
    print 'key=%s, adj=%s, r=%s\n-------\n'%(key,adj,r)
    act(key,adj,r, tdf)

In [61]:
decipher("latest balance", df)

balance NN ROOT
 +-- latest JJS amod
-----
in last month
From 06/12/16 00:00:00 to 07/12/16 00:00:00
key=balance, adj=curr, r=From 06/12/16 00:00:00 to 07/12/16 00:00:00
-------

Current Balance: 95746.70


In [63]:
decipher("average balance from august 2015 to june 2016", df)

balance NN ROOT
 +-- average JJ amod
 +-- from IN prep
     +-- 2015 CD pobj
     |   +-- august JJ amod
     +-- to IN prep
         +-- june NN pobj
             +-- 2016 CD num
-----
Tree form
 from
	 2015
		 august
	 to
		 june
			 2016
-----
2015 august to june 2016
from 2015 august to june 2016
From 08/01/15 00:00:00 to 06/01/16 00:00:00
key=balance, adj=avg, r=From 08/01/15 00:00:00 to 06/01/16 00:00:00
-------

Avg Balance for Timeline: From 08/01/15 00:00:00 to 06/01/16 00:00:00: 58625.19


In [30]:
r.start.year

2016

In [32]:
td = datetime.date(r.start.year, r.start.month, r.start.day)

In [34]:
start = datetime.date(r.start.year, r.start.month, r.start.day)
end = datetime.date(r.end.year, r.end.month, r.end.day)

In [35]:
start

datetime.date(2016, 6, 12)

In [36]:
end

datetime.date(2016, 7, 12)

In [44]:
df[start:end].Balance.mean()

93440.32857142854